In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
import pickle

In [8]:
def load_artifact(targ_file:str):
    targ_path = os.path.join(dir,targ_file)

    with open(targ_path,'rb') as fp:
        test_artifact = pickle.load(fp)

    return test_artifact

In [9]:
# Each time that you initialize Dagster a new temporary folder is created.
# For the porpuse of this exercise we will save data in that temporary folder.
# Substitute tmp*** with your folder (you can fin it in this repo).
dir = "../dagster_home/storage/"

`core_data`

| Feature Name           | Description                                                  |
|------------------------|--------------------------------------------------------------|
| rating_account_id      | Unique identifier for the contract account                    |
| customer_id            | Unique identifier for the customer                           |
| age                    | Age of the customer                                          |
| contract_lifetime_days | Total duration of the customer contract in days              |
| remaining_binding_days | Number of days left in the contract binding period - usual binding period is 2 years        |
| has_special_offer      | Indicates if the customer has a special offer      |
| is_magenta1_customer   | Indicates if the customer is part of the Magenta1 program - fedelty program    |
| available_gb           | Amount of mobile data included in the current tariff         |
| gross_mrc              | Gross monthly recurring charge (in euros)                    |
| smartphone_brand       | Brand of the customer’s smartphone                           |
| has_done_upselling     | Whether the customer has already done an upsell in the last 3 years      |


In [11]:
core_data = load_artifact("core_data")
core_data

,rating_account_id,customer_id,age,contract_lifetime_days,remaining_binding_days,has_special_offer,is_magenta1_customer,available_gb,gross_mrc,smartphone_brand,has_done_upselling
0,289094,4.161115,36,878,325,0,0,20,70.00,iPhone,0
1,677626,2.429976,34,998,614,0,0,0,5.00,Samsung,0
2,769928,3.875044,36,37,-26,0,1,50,16.94,Samsung,0
3,873260,4.649933,50,503,-149,0,1,20,30.20,iPhone,1
4,109774,3.851059,47,331,-328,1,1,None,46.12,Samsung,0
...,...,...,...,...,...,...,...,...,...,...,...
99995,502283,5.605022,88,1573,-576,0,0,10,34.18,Samsung,0
99996,618421,2.862063,85,1138,412,1,0,40,50.10,iPhone,0
99997,104422,2.414264,79,1709,-494,0,0,10,12.96,Samsung,0
99998,642380,3.619106,84,1592,403,0,0,10,56.73,Samsung,0


`usage_info`

| Feature Name           | Description                                                  |
|------------------------|--------------------------------------------------------------|
| rating_account_id      | Unique identifier for the contract account                    |
| billed_period_month_d  | Billing period (monthly)                                     |
| has_used_roaming       | Indicates if roaming was used during the period            |
| used_gb                | Amount of mobile data used in the billing period (in GB)     |


In [6]:
usage_info = load_artifact("usage_info")
usage_info

,rating_account_id,billed_period_month_d,has_used_roaming,used_gb
0,356596,2024-04-01,1,0.6
1,356596,2024-05-01,0,1.0
2,356596,2024-06-01,0,1.0
3,356596,2024-07-01,0,0.2
4,417472,2024-04-01,1,0.7
...,...,...,...,...
399995,936250,2024-07-01,1,49.8
399996,686754,2024-04-01,0,58.4
399997,686754,2024-05-01,0,23.8
399998,686754,2024-06-01,0,43.7


`customer_interactions`

| Feature Name   | Description                                                              |
|----------------|--------------------------------------------------------------------------|
| customer_id    | Unique identifier for the customer                                       |
| type_subtype   | Category and subtype of the interaction (e.g., tariff change, billing)   |
| n              | Number of interactions of this type in the last 6 months                                |
| days_since_last| Number of days since the last interaction of this type                   |


In [7]:
customer_interactions = load_artifact("customer_interactions")
customer_interactions

,customer_id,type_subtype,n,days_since_last
0,1.827889,produkte&services-tarifdetails,1,148
1,1.827889,rechnungsanfragen,1,41
2,4.507001,produkte&services-tarifwechsel,3,0
3,4.408125,rechnungsanfragen,2,164
4,1.123819,produkte&services-tarifdetails,1,141
...,...,...,...,...
63157,3.874483,rechnungsanfragen,1,80
63158,1.666575,produkte&services-tarifdetails,2,87
63159,1.666575,rechnungsanfragen,4,137
63160,5.306865,rechnungsanfragen,2,109
